<a href="https://colab.research.google.com/github/chengsh7/tutorials/blob/master/sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This code is a prototype and not engineered for production use. 
# Error handling is incomplete or inappropriate for usage beyond 
# a development sample.

# Please include the above at the top of each file

# This is a Data Analytics Demo

# Media Prima: Vertex0 Refined: UseCase3 (NEW)

In [2]:
# Import
import os

from google.cloud import bigquery
from google.cloud.bigquery.schema import SchemaField


import pandas

In [9]:
IS_DEBUG = True

project_id = "trainer-sh"

print(f"IS_DEBUG: {IS_DEBUG}")
print(f"Project: {project_id}")

IS_DEBUG: True
Project: trainer-sh


In [4]:
# Authentication

if IS_DEBUG:
  from google.colab import auth
  auth.authenticate_user()
  print('Authenticated')
  
  client = bigquery.Client(project=project_id)
else:
#   import inspect
#   current_path = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
#   key_file_path = "./keys/mp-vertex-f69c5b2855cf.json"
#   key_file_path = f"{current_path}/{key_file_path}"
#   client = bigquery.Client.from_service_account_json(key_file_path)
    client = bigquery.Client(project=project_id)

Authenticated


## Refined Zone

Create tables based on model

### Initialization

In [5]:
dataset_id = "cpb101_flight_data"

dataset = client.dataset(dataset_id, project_id)

print(f"REFINED Dataset: {dataset_id}")

REFINED Dataset: aging


### debtor_aging_fms

In [6]:
# destination table
table_name = "debtor_aging_fms"
print(f"Destination Table Name: {dataset_id}.{table_name}")

Destination Table Name: aging.debtor_aging_fms


In [7]:
sql = f"""
  WITH t1 AS (
      SELECT
        'NSTP' platform,
        agesum.AGING_DATE aging_date,
        EXTRACT(YEAR FROM agesum.AGING_DATE) AS aging_year,
        FORMAT_DATE("%Y-%m", agesum.AGING_DATE) AS aging_month,
        FORMAT_DATE("%Y-Q%Q", agesum.AGING_DATE) AS aging_quarter,  
        agesum.INDUSTRY industry,
        agesum.TERRITORY territory,
        agesum.GROUPING `grouping`,
        agesum.collector collector,    
        agesum.cust_no customer_no,
        cu.customer_name customer_name,    
        agesum.amt_due bal_amount,
        agesum.b0_amt b0 ,
        agesum.b1_amt b1,
        agesum.b2_amt b2,
        agesum.b3_amt b3,
        agesum.b4_amt b4,
        agesum.b5_amt b5,
        agesum.b6_amt b6,
        agesum.cd_amt cash_deposit,
        agesum.bg_amt bank_guarantee
      FROM `mp-vertex.trusted_oraclefms.xxar_agehist_summary` agesum
        INNER JOIN `mp-vertex.trusted_oraclefms.ar_customers` cu
          ON agesum.CUST_NO = cu.customer_number),
      t2 AS (
        SELECT
          ROW_NUMBER() OVER (PARTITION BY customer_no, aging_month ORDER BY aging_date DESC) AS ROW_ID, 
          platform,
          aging_date,
          aging_year,
          aging_month,
          aging_quarter,
          industry,
          territory,
          `grouping`,
          collector,    
          customer_no,
          customer_name,
          bal_amount,
          b0,
          b1,
          b2,
          b3,
          b4,
          b5,
          b6,
          cash_deposit,
          bank_guarantee
        FROM t1
      )
      
      SELECT * EXCEPT(ROW_ID) FROM t2
        WHERE ROW_ID = 1
        ORDER BY aging_date DESC, customer_no;
    """
print(sql)


  WITH t1 AS (
      SELECT
        'NSTP' platform,
        agesum.AGING_DATE aging_date,
        EXTRACT(YEAR FROM agesum.AGING_DATE) AS aging_year,
        FORMAT_DATE("%Y-%m", agesum.AGING_DATE) AS aging_month,
        FORMAT_DATE("%Y-Q%Q", agesum.AGING_DATE) AS aging_quarter,  
        agesum.INDUSTRY industry,
        agesum.TERRITORY territory,
        agesum.GROUPING `grouping`,
        agesum.collector collector,    
        agesum.cust_no customer_no,
        cu.customer_name customer_name,    
        agesum.amt_due bal_amount,
        agesum.b0_amt b0 ,
        agesum.b1_amt b1,
        agesum.b2_amt b2,
        agesum.b3_amt b3,
        agesum.b4_amt b4,
        agesum.b5_amt b5,
        agesum.b6_amt b6,
        agesum.cd_amt cash_deposit,
        agesum.bg_amt bank_guarantee
      FROM `mp-vertex.trusted_oraclefms.xxar_agehist_summary` agesum
        INNER JOIN `mp-vertex.trusted_oraclefms.ar_customers` cu
          ON agesum.CUST_NO = cu.customer_number),
      t2 AS (

In [ ]:
# destination Table
destination_table = dataset.table(table_name)

print(f"Destination Table: {dataset.dataset_id}.{table_name}")

job_config = bigquery.QueryJobConfig()
job_config.destination = destination_table
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

try:
  # Start the query, passing in the extra configuration.
  query_job = client.query(
      sql,
      job_config=job_config)  # API request - starts the query

  query_job.result()  # Waits for the query to finish
  assert query_job.state == "DONE"

  print("Job finished")

  print(f'Query results loaded to table {dataset_id}.{table_name}')
except Exception as e:
  print(e)
  raise ValueError(f"Fail to load: {dataset_id}.{table_name}")

Destination Table: aging.debtor_aging_fms
Job finished
Query results loaded to table aging.debtor_aging_fms


In [8]:
if IS_DEBUG:
  sql = (
      f"""
        SELECT * FROM `{dataset_id}.{table_name}` 
        LIMIT 5
      """)
  print(sql)
  df = client.query(sql).to_dataframe()
  print(df)


        SELECT * FROM `aging.debtor_aging_fms` 
        LIMIT 5
      


Forbidden: ignored

### debtor_aging_coda


In [ ]:
# destination table
table_name = "debtor_aging_coda"
print(f"Destination Table Name: {dataset_id}.{table_name}")

Destination Table Name: aging.debtor_aging_coda


In [ ]:
sql = f"""
      WITH 
      t0 AS(
        SELECT '2019-07-31' AS aging_date
      ),
      t1 AS (
      SELECT 
        CASE WHEN head.cmpcode IN ('TV3LIVE', '8TV LIVE', 'NTV7', 'TV9LIVE') THEN 'TVN' 
             WHEN head.cmpcode IN ('HOTFM', 'MAXLIVE', 'KOOLFM', 'ONEFM') then 'RIPPLE'
             END AS platform,
        PARSE_DATE('%Y-%m-%d', (SELECT aging_date FROM t0)) AS aging_date,
        DATE(head.docdate) AS doc_date,
        CAST(line.valuehome AS NUMERIC) AS amount,
        line.el3 AS agency_no,
        e3.name AS agency_name,
        line.el4 AS customer_no,
        e4.name AS customer_name
      FROM `mp-vertex.trusted_coda.oas_dochead_inv` head
        LEFT JOIN `mp-vertex.trusted_coda.oas_docline_inv` line 
          ON head.cmpcode  = line.cmpcode AND head.doccode = line.doccode AND head.docnum = line.docnum
        LEFT JOIN `mp-vertex.trusted_coda.oas_element_3` e3
          ON line.el3 = e3.code AND line.cmpcode = e3.cmpcode
        LEFT JOIN `mp-vertex.trusted_coda.oas_element_4` e4
          ON line.el4 = e4.code AND line.cmpcode = e4.cmpcode 
      WHERE
        head.doccode = 'AINV' 
        AND line.statpay = 84
        AND line.el2 = '53001'),

        t2 AS (
          SELECT
            platform,
            aging_date,
            DATE_DIFF(aging_date, doc_date, DAY) as aging,
            agency_no,
            agency_name,
            customer_no,
            customer_name,
            amount
          FROM t1
        ),
        t3 AS (
          SELECT t2.* EXCEPT(aging, agency_no, agency_name),
          CASE 
            WHEN aging < 31 THEN "b0"
            WHEN aging < 61 THEN "b1"
            WHEN aging < 91 THEN "b2"
            WHEN aging <121 THEN "b3"
            WHEN aging <151 THEN "b4"
            WHEN aging <181 THEN "b5"
          ELSE
            "b6"
          END AS bucket_id,
          CASE 
            WHEN REGEXP_CONTAINS(agency_name, r'/DIRECT CLIENT/') THEN "DIRECT ACCOUNT"
          ELSE
            "AGENCY"
          END AS industry
          FROM t2
        ),
        t4 AS (
          SELECT
          platform,
          aging_date,
          industry,
          customer_no, 
          customer_name, 
          bucket_id,
          SUM(amount) AS bal_amount
          FROM t3
          GROUP BY platform, aging_date, industry, customer_no, customer_name, bucket_id
        ),
        t5 AS (
          SELECT
          platform,
          aging_date,
          industry,
          customer_no, 
          customer_name,
          MAX(IF(bucket_id = 'b0', bal_amount, 0)) AS b0,
          MAX(IF(bucket_id = 'b1', bal_amount, 0)) AS b1,
          MAX(IF(bucket_id = 'b2', bal_amount, 0)) AS b2,
          MAX(IF(bucket_id = 'b3', bal_amount, 0)) AS b3,
          MAX(IF(bucket_id = 'b4', bal_amount, 0)) AS b4,
          MAX(IF(bucket_id = 'b5', bal_amount, 0)) AS b5,
          MAX(IF(bucket_id = 'b6', bal_amount, 0)) AS b6
        FROM t4
        GROUP BY platform, aging_date, industry, customer_no, customer_name
        ),
        t6 AS (
          SELECT
            platform,
            aging_date,
            EXTRACT(YEAR FROM aging_date) AS aging_year,
            FORMAT_DATE("%Y-%m", aging_date) AS aging_month,
            FORMAT_DATE("%Y-Q%Q", aging_date) AS aging_quarter,
            industry,
            'UNKNOWN' AS territory,
            'UNKNOWN' AS `grouping`,
            'UNKNOWN' AS `collector`,
            customer_no, 
            customer_name,
            (b0 + b1 + b2 + b3 + b4 + b5 + b6) AS bal_amount,
            b0,
            b1,
            b2,
            b3,
            b4,
            b5,
            b6,
            0 AS cash_deposit,
            0 AS bank_guarantee
          FROM t5
        )
        SELECT * FROM t6;

    """
print(sql)


      WITH 
      t0 AS(
        SELECT '2019-07-31' AS aging_date
      ),
      t1 AS (
      SELECT 
        CASE WHEN head.cmpcode IN ('TV3LIVE', '8TV LIVE', 'NTV7', 'TV9LIVE') THEN 'TVN' 
             WHEN head.cmpcode IN ('HOTFM', 'MAXLIVE', 'KOOLFM', 'ONEFM') then 'RIPPLE'
             END AS platform,
        PARSE_DATE('%Y-%m-%d', (SELECT aging_date FROM t0)) AS aging_date,
        DATE(head.docdate) AS doc_date,
        CAST(line.valuehome AS NUMERIC) AS amount,
        line.el3 AS agency_no,
        e3.name AS agency_name,
        line.el4 AS customer_no,
        e4.name AS customer_name
      FROM `mp-vertex.trusted_coda.oas_dochead_inv` head
        LEFT JOIN `mp-vertex.trusted_coda.oas_docline_inv` line 
          ON head.cmpcode  = line.cmpcode AND head.doccode = line.doccode AND head.docnum = line.docnum
        LEFT JOIN `mp-vertex.trusted_coda.oas_element_3` e3
          ON line.el3 = e3.code AND line.cmpcode = e3.cmpcode
        LEFT JOIN `mp-vertex.trusted_coda.oas

In [ ]:
# destination Table
destination_table = dataset.table(table_name)

print(f"Destination Table: {dataset.dataset_id}.{table_name}")

job_config = bigquery.QueryJobConfig()
job_config.destination = destination_table
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

try:
  # Start the query, passing in the extra configuration.
  query_job = client.query(
      sql,
      job_config=job_config)  # API request - starts the query

  query_job.result()  # Waits for the query to finish
  assert query_job.state == "DONE"

  print("Job finished")

  print(f'Query results loaded to table {dataset_id}.{table_name}')
except Exception as e:
  print(e)
  raise ValueError(f"Fail to load: {dataset_id}.{table_name}")

Destination Table: aging.debtor_aging_coda
Job finished
Query results loaded to table aging.debtor_aging_coda


In [ ]:
if IS_DEBUG:
  sql = (
      f"""
        SELECT * FROM `{dataset_id}.{table_name}` 
        LIMIT 5
      """)
  print(sql)
  df = client.query(sql).to_dataframe()
  print(df)


        SELECT * FROM `aging.debtor_aging_coda` 
        LIMIT 5
      
  platform  aging_date  aging_year  ...         b6 cash_deposit bank_guarantee
0      TVN  2019-07-31        2019  ...     284716            0              0
1      TVN  2019-07-31        2019  ...  637861.71            0              0
2      TVN  2019-07-31        2019  ...          0            0              0
3      TVN  2019-07-31        2019  ...          0            0              0
4      TVN  2019-07-31        2019  ...          0            0              0

[5 rows x 21 columns]


### debtor_aging

In [ ]:
# destination table
table_name = "debtor_aging"
print(f"Destination Table Name: {dataset_id}.{table_name}")

Destination Table Name: aging.debtor_aging


In [ ]:
sql = f"""
    WITH
    fms AS (
      SELECT
        platform,
        aging_date,
        aging_year,
        aging_month,
        aging_quarter,
        industry,
        territory,
        `grouping`,
        collector,    
        customer_no,
        customer_name,
        bal_amount,
        b0,
        b1,
        b2,
        b3,
        b4,
        b5,
        b6,
        cash_deposit,
        bank_guarantee
      FROM
        `groupfinancial.aging.debtor_aging_fms`
    ),
    coda AS (
      SELECT
        platform,
        aging_date,
        aging_year,
        aging_month,
        aging_quarter,
        industry,
        territory,
        `grouping`,
        collector,    
        customer_no,
        customer_name,
        bal_amount,
        b0,
        b1,
        b2,
        b3,
        b4,
        b5,
        b6,
        cash_deposit,
        bank_guarantee
      FROM
        `groupfinancial.aging.debtor_aging_coda`
    ),
    t1 AS (
      SELECT * FROM fms UNION ALL
      SELECT * FROM coda
    ),
    t2 AS (
      SELECT
        platform,
        aging_date,
        aging_year,
        aging_month,
        aging_quarter,
        industry,
        territory,
        `grouping`,
        collector,    
        customer_no,
        customer_name,
        bal_amount,
        b0,
        b1,
        b2,
        b3,
        b4,
        b5,
        b6,
        LAG(bal_amount) OVER (PARTITION BY customer_no ORDER BY aging_month ASC) AS p_bal_amount,
        LAG(b0) OVER (PARTITION BY customer_no ORDER BY aging_month ASC) AS p_b0,
        LAG(b1) OVER (PARTITION BY customer_no ORDER BY aging_month ASC) AS p_b1,
        LAG(b2) OVER (PARTITION BY customer_no ORDER BY aging_month ASC) AS p_b2,
        LAG(b3) OVER (PARTITION BY customer_no ORDER BY aging_month ASC) AS p_b3,
        LAG(b4) OVER (PARTITION BY customer_no ORDER BY aging_month ASC) AS p_b4,
        LAG(b5) OVER (PARTITION BY customer_no ORDER BY aging_month ASC) AS p_b5,
        LAG(b6) OVER (PARTITION BY customer_no ORDER BY aging_month ASC) AS p_b6,
        cash_deposit,
        bank_guarantee
      FROM
        t1     
    )      
      
    SELECT *
    FROM t2 
    -- WHERE customer_no LIKE '%1669%'
    ORDER BY aging_month DESC, customer_no;
    """
print(sql)


    WITH
    fms AS (
      SELECT
        platform,
        aging_date,
        aging_year,
        aging_month,
        aging_quarter,
        industry,
        territory,
        `grouping`,
        collector,    
        customer_no,
        customer_name,
        bal_amount,
        b0,
        b1,
        b2,
        b3,
        b4,
        b5,
        b6,
        cash_deposit,
        bank_guarantee
      FROM
        `groupfinancial.aging.debtor_aging_fms`
    ),
    coda AS (
      SELECT
        platform,
        aging_date,
        aging_year,
        aging_month,
        aging_quarter,
        industry,
        territory,
        `grouping`,
        collector,    
        customer_no,
        customer_name,
        bal_amount,
        b0,
        b1,
        b2,
        b3,
        b4,
        b5,
        b6,
        cash_deposit,
        bank_guarantee
      FROM
        `groupfinancial.aging.debtor_aging_coda`
    ),
    t1 AS (
      SELECT * FROM fms UNION ALL
      SE

In [ ]:
# destination Table
destination_table = dataset.table(table_name)

print(f"Destination Table: {dataset.dataset_id}.{table_name}")

job_config = bigquery.QueryJobConfig()
job_config.destination = destination_table
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

try:
  # Start the query, passing in the extra configuration.
  query_job = client.query(
      sql,
      job_config=job_config)  # API request - starts the query

  query_job.result()  # Waits for the query to finish
  assert query_job.state == "DONE"

  print("Job finished")

  print(f'Query results loaded to table {dataset_id}.{table_name}')
except Exception as e:
  print(e)
  raise ValueError(f"Fail to load: {dataset_id}.{table_name}")

Destination Table: aging.debtor_aging
Job finished
Query results loaded to table aging.debtor_aging


In [ ]:
if IS_DEBUG:
  sql = (
      f"""
        SELECT * FROM `{dataset_id}.{table_name}` 
        LIMIT 5
      """)
  print(sql)
  df = client.query(sql).to_dataframe()
  print(df)


        SELECT * FROM `aging.debtor_aging` 
        LIMIT 5
      
  platform  aging_date  aging_year  ... p_b6 cash_deposit bank_guarantee
0     NSTP  2019-07-31        2019  ...    0            0         100000
1     NSTP  2019-07-31        2019  ...    0            0          50000
2     NSTP  2019-07-31        2019  ...    0            0          50000
3     NSTP  2019-07-31        2019  ...    0            0          50000
4     NSTP  2019-07-31        2019  ...    0            0          50000

[5 rows x 29 columns]


### debtor_aging_by_quarter

In [ ]:
# destination table
table_name = "debtor_aging_by_quarter"
print(f"Destination Table Name: {dataset_id}.{table_name}")

Destination Table Name: aging.debtor_aging_by_quarter


In [ ]:
sql = f"""
    WITH
    t0 AS (
      SELECT * 
      FROM
        `groupfinancial.aging.debtor_aging`
    ),
    t1 AS (
      SELECT
        ROW_NUMBER() OVER (PARTITION BY customer_no, aging_quarter ORDER BY aging_month DESC) AS ROW_ID,
        platform,
        aging_date,
        aging_year,
        aging_month,
        aging_quarter,
        industry,
        territory,
        `grouping`,
        collector,    
        customer_no,
        customer_name,
        bal_amount,
        b0,
        b1,
        b2,
        b3,
        b4,
        b5,
        b6
      FROM
        t0
    ),
    t2 AS(
      SELECT * EXCEPT (ROW_ID)
      FROM t1
      WHERE ROW_ID = 1
    ),
    t3 AS (
      SELECT
        platform,
        aging_quarter,
        industry,
        territory,
        `grouping`,
        collector,    
        customer_no,
        customer_name,
        bal_amount,
        b0,
        b1,
        b2,
        b3,
        b4,
        b5,
        b6,
        LAG(bal_amount) OVER (PARTITION BY customer_no ORDER BY aging_quarter ASC) AS p_bal_amount,
        LAG(b0) OVER (PARTITION BY customer_no ORDER BY aging_quarter ASC) AS p_b0,
        LAG(b1) OVER (PARTITION BY customer_no ORDER BY aging_quarter ASC) AS p_b1,
        LAG(b2) OVER (PARTITION BY customer_no ORDER BY aging_quarter ASC) AS p_b2,
        LAG(b0) OVER (PARTITION BY customer_no ORDER BY aging_quarter ASC) AS p_b3,
        LAG(b0) OVER (PARTITION BY customer_no ORDER BY aging_quarter ASC) AS p_b4,
        LAG(b0) OVER (PARTITION BY customer_no ORDER BY aging_quarter ASC) AS p_b5,
        LAG(b6) OVER (PARTITION BY customer_no ORDER BY aging_quarter ASC) AS p_b6
      FROM
        t2
    ),
    t4 AS (
      SELECT 
        t0.platform,
        t0.aging_date,
        t0.industry,
        t0.territory,
        t0.`grouping`,
        t0.collector,    
        t0.customer_no,
        t0.customer_name,
        t3.bal_amount,
        t3.b0,
        t3.b1,
        t3.b2,
        t3.b3,
        t3.b4,
        t3.b5,
        t3.b6,
        t3.p_b0,
        t3.p_b1,
        t3.p_b2,
        t3.p_b3,
        t3.p_b4,
        t3.p_b5,
        t3.p_b6
      FROM t0 LEFT JOIN t3 
        ON t0.platform = t3.platform AND t0.aging_quarter = t3.aging_quarter AND t0.customer_no = t3.customer_no
    )
    SELECT *
    FROM t4
    -- WHERE customer_no LIKE '%1669%'
    ORDER BY aging_date DESC, customer_no;
    
    """
print(sql)


    WITH
    t0 AS (
      SELECT * 
      FROM
        `groupfinancial.aging.debtor_aging`
    ),
    t1 AS (
      SELECT
        ROW_NUMBER() OVER (PARTITION BY customer_no, aging_quarter ORDER BY aging_month DESC) AS ROW_ID,
        platform,
        aging_date,
        aging_year,
        aging_month,
        aging_quarter,
        industry,
        territory,
        `grouping`,
        collector,    
        customer_no,
        customer_name,
        bal_amount,
        b0,
        b1,
        b2,
        b3,
        b4,
        b5,
        b6
      FROM
        t0
    ),
    t2 AS(
      SELECT * EXCEPT (ROW_ID)
      FROM t1
      WHERE ROW_ID = 1
    ),
    t3 AS (
      SELECT
        platform,
        aging_quarter,
        industry,
        territory,
        `grouping`,
        collector,    
        customer_no,
        customer_name,
        bal_amount,
        b0,
        b1,
        b2,
        b3,
        b4,
        b5,
        b6,
        LAG(bal_amount) OVER (PA

In [ ]:
# destination Table
destination_table = dataset.table(table_name)

print(f"Destination Table: {dataset.dataset_id}.{table_name}")

job_config = bigquery.QueryJobConfig()
job_config.destination = destination_table
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

try:
  # Start the query, passing in the extra configuration.
  query_job = client.query(
      sql,
      job_config=job_config)  # API request - starts the query

  query_job.result()  # Waits for the query to finish
  assert query_job.state == "DONE"

  print("Job finished")

  print(f'Query results loaded to table {dataset_id}.{table_name}')
except Exception as e:
  print(e)
  raise ValueError(f"Fail to load: {dataset_id}.{table_name}")

Destination Table: aging.debtor_aging_by_quarter
Job finished
Query results loaded to table aging.debtor_aging_by_quarter


In [ ]:
if IS_DEBUG:
  sql = (
      f"""
        SELECT * FROM `{dataset_id}.{table_name}` 
        LIMIT 5
      """)
  print(sql)
  df = client.query(sql).to_dataframe()
  print(df)


        SELECT * FROM `aging.debtor_aging_by_quarter` 
        LIMIT 5
      
  platform  aging_date industry  ...      p_b4      p_b5 p_b6
0     NSTP  2019-07-31   AGENCY  ...  64822.83  64822.83    0
1     NSTP  2019-07-31   AGENCY  ...         0         0    0
2     NSTP  2019-07-31   AGENCY  ...    7102.6    7102.6    0
3     NSTP  2019-07-31   AGENCY  ...  71653.98  71653.98    0
4     NSTP  2019-07-31   AGENCY  ...  87590.79  87590.79    0

[5 rows x 23 columns]


### debtor_aging_by_bucket

In [ ]:
# destination table
table_name = "debtor_aging_by_bucket"
print(f"Destination Table Name: {dataset_id}.{table_name}")

Destination Table Name: aging.debtor_aging_by_bucket


In [ ]:
sql = f"""
      
      WITH
      t1 AS (
      SELECT
      platform,
      aging_date,
      industry,
      territory,
      `grouping`,
      collector,    
      customer_no,
      customer_name,
      [
        STRUCT('b0' AS bucket_id, '  1- 30 days' AS bucket, b0 AS amount, p_b0 AS prev_amount),
        STRUCT('b1' AS bucket_id, ' 31- 60 days' AS bucket, b1 AS amount, p_b1 AS prev_amount),
        STRUCT('b2' AS bucket_id, ' 61- 90 days' AS bucket, b2 AS amount, p_b2 AS prev_amount),
        STRUCT('b3' AS bucket_id, ' 91-120 days' AS bucket, b3 AS amount, p_b3 AS prev_amount),
        STRUCT('b4' AS bucket_id, '121-150 days' AS bucket, b4 AS amount, p_b4 AS prev_amount),
        STRUCT('b5' AS bucket_id, '151-180 days' AS bucket, b5 AS amount, p_b5 AS prev_amount),
        STRUCT('b6' AS bucket_id, '  > 180 days' AS bucket, b6 AS amount, p_b6 AS prev_amount)
      ] AS metrics_data
      FROM
      `groupfinancial.aging.debtor_aging` 
      ),
      t2 AS (
      SELECT
      platform,
      aging_date,
      industry,
      territory,
      `grouping`,
      collector,    
      customer_no,
      customer_name,
      metrics_data
      FROM
      t1
      CROSS JOIN UNNEST(t1.metrics_data) AS metrics_data)

      SELECT 
      platform,
      aging_date,
      industry,
      territory,
      `grouping`,
      collector,    
      customer_no,
      customer_name,
      metrics_data.bucket_id AS bucket_id,
      metrics_data.bucket AS bucket,
      CAST(metrics_data.amount AS NUMERIC) AS amount,
      CAST(metrics_data.prev_amount AS NUMERIC) AS prev_amount,
      CASE WHEN SIGN(metrics_data.amount) < 0
        THEN 0
        ELSE metrics_data.amount
      END AS net_amount,
      CASE WHEN SIGN(metrics_data.prev_amount) < 0
        THEN 0
        ELSE metrics_data.prev_amount
      END AS net_prev_amount
      FROM t2
      ORDER BY customer_no, aging_date DESC, bucket_id;
    
    """
print(sql)


      
      WITH
      t1 AS (
      SELECT
      platform,
      aging_date,
      industry,
      territory,
      `grouping`,
      collector,    
      customer_no,
      customer_name,
      [
        STRUCT('b0' AS bucket_id, '  1- 30 days' AS bucket, b0 AS amount, p_b0 AS prev_amount),
        STRUCT('b1' AS bucket_id, ' 31- 60 days' AS bucket, b1 AS amount, p_b1 AS prev_amount),
        STRUCT('b2' AS bucket_id, ' 61- 90 days' AS bucket, b2 AS amount, p_b2 AS prev_amount),
        STRUCT('b3' AS bucket_id, ' 91-120 days' AS bucket, b3 AS amount, p_b3 AS prev_amount),
        STRUCT('b4' AS bucket_id, '121-150 days' AS bucket, b4 AS amount, p_b4 AS prev_amount),
        STRUCT('b5' AS bucket_id, '151-180 days' AS bucket, b5 AS amount, p_b5 AS prev_amount),
        STRUCT('b6' AS bucket_id, '  > 180 days' AS bucket, b6 AS amount, p_b6 AS prev_amount)
      ] AS metrics_data
      FROM
      `groupfinancial.aging.debtor_aging` 
      ),
      t2 AS (
      SELECT
      platform,

In [ ]:
# destination Table
destination_table = dataset.table(table_name)

print(f"Destination Table: {dataset.dataset_id}.{table_name}")

job_config = bigquery.QueryJobConfig()
job_config.destination = destination_table
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

try:
  # Start the query, passing in the extra configuration.
  query_job = client.query(
      sql,
      job_config=job_config)  # API request - starts the query

  query_job.result()  # Waits for the query to finish
  assert query_job.state == "DONE"

  print("Job finished")

  print(f'Query results loaded to table {dataset_id}.{table_name}')
except Exception as e:
  print(e)
  raise ValueError(f"Fail to load: {dataset_id}.{table_name}")

Destination Table: aging.debtor_aging_by_bucket
Job finished
Query results loaded to table aging.debtor_aging_by_bucket


In [ ]:
if IS_DEBUG:
  sql = (
      f"""
        SELECT * FROM `{dataset_id}.{table_name}` 
        LIMIT 5
      """)
  print(sql)
  df = client.query(sql).to_dataframe()
  print(df)


        SELECT * FROM `aging.debtor_aging_by_bucket` 
        LIMIT 5
      
  platform  aging_date industry  ... prev_amount net_amount net_prev_amount
0     NSTP  2015-04-30   AGENCY  ...           0          0               0
1     NSTP  2015-04-30   AGENCY  ...           0          0               0
2     NSTP  2015-04-30   AGENCY  ...      836.13          0          836.13
3     NSTP  2015-04-30   AGENCY  ...           0     836.13               0
4     NSTP  2015-04-30   AGENCY  ...           0          0               0

[5 rows x 14 columns]


### debtor_aging_by_bucket_quarter

In [ ]:
# destination table
table_name = "debtor_aging_by_bucket_quarter"
print(f"Destination Table Name: {dataset_id}.{table_name}")

Destination Table Name: aging.debtor_aging_by_bucket_quarter


In [ ]:
sql = f"""
      
      WITH
      t1 AS (
      SELECT
      platform,
      aging_date,
      industry,
      territory,
      `grouping`,
      collector,    
      customer_no,
      customer_name,
      [
        STRUCT('b0' AS bucket_id, '  1- 30 days' AS bucket, b0 AS amount, p_b0 AS prev_amount),
        STRUCT('b1' AS bucket_id, ' 31- 60 days' AS bucket, b1 AS amount, p_b1 AS prev_amount),
        STRUCT('b2' AS bucket_id, ' 61- 90 days' AS bucket, b2 AS amount, p_b2 AS prev_amount),
        STRUCT('b3' AS bucket_id, ' 91-120 days' AS bucket, b3 AS amount, p_b3 AS prev_amount),
        STRUCT('b4' AS bucket_id, '121-150 days' AS bucket, b4 AS amount, p_b4 AS prev_amount),
        STRUCT('b5' AS bucket_id, '151-180 days' AS bucket, b5 AS amount, p_b5 AS prev_amount),
        STRUCT('b6' AS bucket_id, '  > 180 days' AS bucket, b6 AS amount, p_b6 AS prev_amount)
      ] AS metrics_data
      FROM
      `groupfinancial.aging.debtor_aging_by_quarter` 
      ),
      t2 AS (
      SELECT
      platform,
      aging_date,
      industry,
      territory,
      `grouping`,
      collector,    
      customer_no,
      customer_name,
      metrics_data
      FROM
      t1
      CROSS JOIN UNNEST(t1.metrics_data) AS metrics_data)

      SELECT 
      platform,
      aging_date,
      industry,
      territory,
      `grouping`,
      collector,    
      customer_no,
      customer_name,
      metrics_data.bucket_id AS bucket_id,
      metrics_data.bucket AS bucket,
      CAST(metrics_data.amount AS NUMERIC) AS amount,
      CAST(metrics_data.prev_amount AS NUMERIC) AS prev_amount,
      CASE WHEN SIGN(metrics_data.amount) < 0
        THEN 0
        ELSE metrics_data.amount
      END AS net_amount,
      CASE WHEN SIGN(metrics_data.prev_amount) < 0
        THEN 0
        ELSE metrics_data.prev_amount
      END AS net_prev_amount
      FROM t2
      ORDER BY customer_no, aging_date DESC, bucket_id;
    
    """
print(sql)


      
      WITH
      t1 AS (
      SELECT
      platform,
      aging_date,
      industry,
      territory,
      `grouping`,
      collector,    
      customer_no,
      customer_name,
      [
        STRUCT('b0' AS bucket_id, '  1- 30 days' AS bucket, b0 AS amount, p_b0 AS prev_amount),
        STRUCT('b1' AS bucket_id, ' 31- 60 days' AS bucket, b1 AS amount, p_b1 AS prev_amount),
        STRUCT('b2' AS bucket_id, ' 61- 90 days' AS bucket, b2 AS amount, p_b2 AS prev_amount),
        STRUCT('b3' AS bucket_id, ' 91-120 days' AS bucket, b3 AS amount, p_b3 AS prev_amount),
        STRUCT('b4' AS bucket_id, '121-150 days' AS bucket, b4 AS amount, p_b4 AS prev_amount),
        STRUCT('b5' AS bucket_id, '151-180 days' AS bucket, b5 AS amount, p_b5 AS prev_amount),
        STRUCT('b6' AS bucket_id, '  > 180 days' AS bucket, b6 AS amount, p_b6 AS prev_amount)
      ] AS metrics_data
      FROM
      `groupfinancial.aging.debtor_aging_by_quarter` 
      ),
      t2 AS (
      SELECT
    

In [ ]:
# destination Table
destination_table = dataset.table(table_name)

print(f"Destination Table: {dataset.dataset_id}.{table_name}")

job_config = bigquery.QueryJobConfig()
job_config.destination = destination_table
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

try:
  # Start the query, passing in the extra configuration.
  query_job = client.query(
      sql,
      job_config=job_config)  # API request - starts the query

  query_job.result()  # Waits for the query to finish
  assert query_job.state == "DONE"

  print("Job finished")

  print(f'Query results loaded to table {dataset_id}.{table_name}')
except Exception as e:
  print(e)
  raise ValueError(f"Fail to load: {dataset_id}.{table_name}")

Destination Table: aging.debtor_aging_by_bucket_quarter
Job finished
Query results loaded to table aging.debtor_aging_by_bucket_quarter


In [ ]:
if IS_DEBUG:
  sql = (
      f"""
        SELECT * FROM `{dataset_id}.{table_name}` 
        LIMIT 5
      """)
  print(sql)
  df = client.query(sql).to_dataframe()
  print(df)


        SELECT * FROM `aging.debtor_aging_by_bucket_quarter` 
        LIMIT 5
      
  platform  aging_date industry  ... prev_amount net_amount net_prev_amount
0     NSTP  2015-04-30   AGENCY  ...           0          0               0
1     NSTP  2015-04-30   AGENCY  ...           0          0               0
2     NSTP  2015-04-30   AGENCY  ...      836.13          0          836.13
3     NSTP  2015-04-30   AGENCY  ...           0     836.13               0
4     NSTP  2015-04-30   AGENCY  ...           0          0               0

[5 rows x 14 columns]


## Report (Authorized View) Zone




### Initialization

In [ ]:
dataset_id = "aging"

dataset = client.dataset(dataset_id, project_id)

print(f"Report Dataset: {dataset_id}")

Report Dataset: aging


### vw_debtor_aging

In [ ]:
# source table
source_dataset_id = "aging"
source_table = "debtor_aging"

# destination view
view_name = f"vw_{source_table}"

print(f"Source Table: {source_dataset_id}.{source_table}")
print(f"View: {dataset_id}.{view_name}")

Source Table: aging.debtor_aging
View: aging.vw_debtor_aging


In [ ]:
sql = f"""
         SELECT t.* 
         FROM `{project_id}.{source_dataset_id}.{source_table}` t
         JOIN `groupfinancial.common.groups_users` g ON SESSION_USER() = g.user
         WHERE t.platform IN (
           CASE
              WHEN g.group='nstp_users' THEN 'NSTP'
              WHEN g.group='bto_users' THEN 'BTO'
              WHEN g.group='tvn_users' THEN 'TVN'
              WHEN g.group='ripple_users' THEN 'RIPPLE'
            END
         )
    """
print(sql)


         SELECT * 
         FROM `groupfinancial.aging.debtor_aging` t
         JOIN `groupfinancial.common.groups_users` g ON SESSION_USER() = g.user
         WHERE t.platform IN (
           CASE
              WHEN g.group='nstp_users' THEN 'NSTP'
              WHEN g.group='bto_users' THEN 'BTO'
              WHEN g.group='tvn_users' THEN 'TVN'
              WHEN g.group='ripple_users' THEN 'RIPPLE'
            END
         )
    


In [ ]:
view = dataset.table(view_name)
table = bigquery.Table(view)
table.view_query = sql
table.view_use_legacy_sql = False

try:
  # delete_table with not_found_ok not working in Airflow bigquery library. 
  # client.delete_table(table, not_found_ok=True)
  try: client.delete_table(table) 
  except: pass

  client.create_table(table)

  print(f'Authorized view created: {dataset_id}.{view_name}')
except:
  raise ValueError(f"Fail to create: {dataset_id}.{view_name}")

Authorized view created: aging.vw_debtor_aging


In [ ]:
if IS_DEBUG:
  sql = (
      f"""
        SELECT * FROM `{dataset_id}.{view_name}` 
        LIMIT 5
      """)
  print(sql)
  df = client.query(sql).to_dataframe()
  print(df)


        SELECT * FROM `aging.vw_debtor_aging` 
        LIMIT 5
      
  platform  aging_date  aging_year  ... p_b6 cash_deposit bank_guarantee
0     NSTP  2019-07-31        2019  ...    0            0         100000
1     NSTP  2019-07-31        2019  ...    0            0          50000
2     NSTP  2019-07-31        2019  ...    0            0          50000
3     NSTP  2019-07-31        2019  ...    0            0          50000
4     NSTP  2019-07-31        2019  ...    0            0          50000

[5 rows x 29 columns]


### vw_debtor_aging_by_quarter

In [ ]:
# source table
source_dataset_id = "aging"
source_table = "debtor_aging_by_quarter"

# destination view
view_name = f"vw_{source_table}"

print(f"Source Table: {source_dataset_id}.{source_table}")
print(f"View: {dataset_id}.{view_name}")

Source Table: aging.debtor_aging_by_quarter
View: aging.vw_debtor_aging_by_quarter


In [ ]:
sql = f"""
      SELECT t.*
      FROM `{project_id}.{source_dataset_id}.{source_table}` t
         JOIN `groupfinancial.common.groups_users` g ON SESSION_USER() = g.user
         WHERE t.platform IN (
           CASE
              WHEN g.group='nstp_users' THEN 'NSTP'
              WHEN g.group='bto_users' THEN 'BTO'
              WHEN g.group='tvn_users' THEN 'TVN'
              WHEN g.group='ripple_users' THEN 'RIPPLE'
            END
         ) 
    """
print(sql)


      SELECT *
      FROM `groupfinancial.aging.debtor_aging_by_quarter` 
    


In [ ]:
view = dataset.table(view_name)
table = bigquery.Table(view)
table.view_query = sql
table.view_use_legacy_sql = False

try:
  # delete_table with not_found_ok not working in Airflow bigquery library. 
  # client.delete_table(table, not_found_ok=True)
  try: client.delete_table(table) 
  except: pass

  client.create_table(table)

  print(f'Authorized view created: {dataset_id}.{view_name}')
except:
  raise ValueError(f"Fail to create: {dataset_id}.{view_name}")

Authorized view created: aging.vw_debtor_aging_by_quarter


In [ ]:
if IS_DEBUG:
  sql = (
      f"""
        SELECT * FROM `{dataset_id}.{view_name}` 
        LIMIT 5
      """)
  print(sql)
  df = client.query(sql).to_dataframe()
  print(df)


        SELECT * FROM `aging.vw_debtor_aging_by_quarter` 
        LIMIT 5
      
  platform  aging_date  aging_year aging_month  ... p_b3 p_b4 p_b5 p_b6
0     NSTP  2019-07-31        2019     2019-07  ...    0    0    0    0
1     NSTP  2019-07-31        2019     2019-07  ...  0.6    0    0    0
2     NSTP  2019-07-31        2019     2019-07  ...    0    0    0    0
3     NSTP  2019-07-31        2019     2019-07  ...    0    0    0    0
4     NSTP  2019-07-31        2019     2019-07  ...    0    0    0    0

[5 rows x 27 columns]


### vw_debtor_aging_history

In [ ]:
# source table
source_dataset_id = "aging"
source_table = "debtor_aging"

# destination view
view_name = f"vw_{source_table}_history"

print(f"Source Table: {source_dataset_id}.{source_table}")
print(f"View: {dataset_id}.{view_name}")

Source Table: aging.debtor_aging
View: aging.vw_debtor_aging_history


In [ ]:
sql = f"""
      SELECT
        platform,
        aging_date AS aging_date_hist,
        aging_year,
        aging_month,
        aging_quarter,
        industry,
        territory,
        `grouping`,
        collector,    
        customer_no,
        customer_name,
        bal_amount,
        b0,
        b1,
        b2,
        b3,
        b4,
        b5,
        b6,
        p_bal_amount,
        p_b0,
        p_b1,
        p_b2,
        p_b3,
        p_b4,
        p_b5,
        p_b6,
        cash_deposit,
        bank_guarantee
      FROM `{project_id}.{source_dataset_id}.{source_table}` t
         JOIN `groupfinancial.common.groups_users` g ON SESSION_USER() = g.user
         WHERE t.platform IN (
           CASE
              WHEN g.group='nstp_users' THEN 'NSTP'
              WHEN g.group='bto_users' THEN 'BTO'
              WHEN g.group='tvn_users' THEN 'TVN'
              WHEN g.group='ripple_users' THEN 'RIPPLE'
            END
         ) 
    """
print(sql)


      SELECT
        platform,
        aging_date AS aging_date_hist,
        aging_year,
        aging_month,
        aging_quarter,
        industry,
        territory,
        `grouping`,
        collector,    
        customer_no,
        customer_name,
        bal_amount,
        b0,
        b1,
        b2,
        b3,
        b4,
        b5,
        b6,
        p_bal_amount,
        p_b0,
        p_b1,
        p_b2,
        p_b3,
        p_b4,
        p_b5,
        p_b6,
        cash_deposit,
        bank_guarantee
      FROM `groupfinancial.aging.debtor_aging` 
    


In [ ]:
view = dataset.table(view_name)
table = bigquery.Table(view)
table.view_query = sql
table.view_use_legacy_sql = False

try:
  # delete_table with not_found_ok not working in Airflow bigquery library. 
  # client.delete_table(table, not_found_ok=True)
  try: client.delete_table(table) 
  except: pass

  client.create_table(table)

  print(f'Authorized view created: {dataset_id}.{view_name}')
except:
  raise ValueError(f"Fail to create: {dataset_id}.{view_name}")

Authorized view created: aging.vw_debtor_aging_history


In [ ]:
if IS_DEBUG:
  sql = (
      f"""
        SELECT * FROM `{dataset_id}.{view_name}` 
        LIMIT 5
      """)
  print(sql)
  df = client.query(sql).to_dataframe()
  print(df)


        SELECT * FROM `aging.vw_debtor_aging_history` 
        LIMIT 5
      
  platform aging_date_hist  aging_year  ... p_b6 cash_deposit bank_guarantee
0     NSTP      2019-07-31        2019  ...    0            0         100000
1     NSTP      2019-07-31        2019  ...    0            0          50000
2     NSTP      2019-07-31        2019  ...    0            0          50000
3     NSTP      2019-07-31        2019  ...    0            0          50000
4     NSTP      2019-07-31        2019  ...    0            0          50000

[5 rows x 29 columns]


### vw_debtor_aging_by_bucket

In [ ]:
# source table
source_dataset_id = "aging"
source_table = "debtor_aging_by_bucket"

# destination view
view_name = f"vw_{source_table}"

print(f"Source Table: {source_dataset_id}.{source_table}")
print(f"View: {dataset_id}.{view_name}")

Source Table: aging.debtor_aging_by_bucket
View: aging.vw_debtor_aging_by_bucket


In [ ]:
sql = f"""
         SELECT t.* 
         FROM `{project_id}.{source_dataset_id}.{source_table}` t
         JOIN `groupfinancial.common.groups_users` g ON SESSION_USER() = g.user
         WHERE t.platform IN (
           CASE
              WHEN g.group='nstp_users' THEN 'NSTP'
              WHEN g.group='bto_users' THEN 'BTO'
              WHEN g.group='tvn_users' THEN 'TVN'
              WHEN g.group='ripple_users' THEN 'RIPPLE'
            END
         ) 
    """
print(sql)


         SELECT * 
         FROM `groupfinancial.aging.debtor_aging_by_bucket` 
    


In [ ]:
view = dataset.table(view_name)
table = bigquery.Table(view)
table.view_query = sql
table.view_use_legacy_sql = False

try:
  # delete_table with not_found_ok not working in Airflow bigquery library. 
  # client.delete_table(table, not_found_ok=True)
  try: client.delete_table(table) 
  except: pass

  client.create_table(table)

  print(f'Authorized view created: {dataset_id}.{view_name}')
except:
  raise ValueError(f"Fail to create: {dataset_id}.{view_name}")

Authorized view created: aging.vw_debtor_aging_by_bucket


In [ ]:
if IS_DEBUG:
  sql = (
      f"""
        SELECT * FROM `{dataset_id}.{view_name}` 
        LIMIT 5
      """)
  print(sql)
  df = client.query(sql).to_dataframe()
  print(df)


        SELECT * FROM `aging.vw_debtor_aging_by_bucket` 
        LIMIT 5
      
  platform  aging_date industry  ... prev_amount net_amount net_prev_amount
0     NSTP  2015-04-30   AGENCY  ...           0          0               0
1     NSTP  2015-04-30   AGENCY  ...           0          0               0
2     NSTP  2015-04-30   AGENCY  ...      836.13          0          836.13
3     NSTP  2015-04-30   AGENCY  ...           0     836.13               0
4     NSTP  2015-04-30   AGENCY  ...           0          0               0

[5 rows x 14 columns]


### vw_debtor_aging_by_bucket_quarter

In [ ]:
# source table
source_dataset_id = "aging"
source_table = "debtor_aging_by_bucket_quarter"

# destination view
view_name = f"vw_{source_table}"

print(f"Source Table: {source_dataset_id}.{source_table}")
print(f"View: {dataset_id}.{view_name}")

Source Table: aging.debtor_aging_by_bucket_quarter
View: aging.vw_debtor_aging_by_bucket_quarter


In [ ]:
sql = f"""
         SELECT t.* 
         FROM `{project_id}.{source_dataset_id}.{source_table}` t
         JOIN `groupfinancial.common.groups_users` g ON SESSION_USER() = g.user
         WHERE t.platform IN (
           CASE
              WHEN g.group='nstp_users' THEN 'NSTP'
              WHEN g.group='bto_users' THEN 'BTO'
              WHEN g.group='tvn_users' THEN 'TVN'
              WHEN g.group='ripple_users' THEN 'RIPPLE'
            END
         ) 
    """
print(sql)


         SELECT * 
         FROM `groupfinancial.aging.debtor_aging_by_bucket_quarter` 
    


In [ ]:
view = dataset.table(view_name)
table = bigquery.Table(view)
table.view_query = sql
table.view_use_legacy_sql = False

try:
  # delete_table with not_found_ok not working in Airflow bigquery library. 
  # client.delete_table(table, not_found_ok=True)
  try: client.delete_table(table) 
  except: pass

  client.create_table(table)

  print(f'Authorized view created: {dataset_id}.{view_name}')
except:
  raise ValueError(f"Fail to create: {dataset_id}.{view_name}")

Authorized view created: aging.vw_debtor_aging_by_bucket_quarter


In [ ]:
if IS_DEBUG:
  sql = (
      f"""
        SELECT * FROM `{dataset_id}.{view_name}` 
        LIMIT 5
      """)
  print(sql)
  df = client.query(sql).to_dataframe()
  print(df)


        SELECT * FROM `aging.vw_debtor_aging_by_bucket_quarter` 
        LIMIT 5
      
  platform  aging_date industry  ... prev_amount net_amount net_prev_amount
0     NSTP  2015-04-30   AGENCY  ...           0          0               0
1     NSTP  2015-04-30   AGENCY  ...           0          0               0
2     NSTP  2015-04-30   AGENCY  ...      836.13          0          836.13
3     NSTP  2015-04-30   AGENCY  ...           0     836.13               0
4     NSTP  2015-04-30   AGENCY  ...           0          0               0

[5 rows x 14 columns]


### vw_debtor_aging_by_bucket_history

In [ ]:
# source table
source_dataset_id = "aging"
source_table = "debtor_aging_by_bucket"

# destination view
view_name = f"vw_{source_table}_history"

print(f"Source Table: {source_dataset_id}.{source_table}")
print(f"View: {dataset_id}.{view_name}")

Source Table: aging.debtor_aging_by_bucket
View: aging.vw_debtor_aging_by_bucket_history


In [ ]:
sql = f"""
        SELECT 
          platform,
          aging_date AS aging_date_hist,
          industry,
          territory,
          `grouping`,
          collector,    
          customer_no,
          customer_name,
          bucket_id,
          bucket,
          amount,
          prev_amount,
          net_amount,
          net_prev_amount
        FROM `{project_id}.{source_dataset_id}.{source_table}` t
        JOIN `groupfinancial.common.groups_users` g ON SESSION_USER() = g.user
        WHERE t.platform IN (
          CASE
             WHEN g.group='nstp_users' THEN 'NSTP'
             WHEN g.group='bto_users' THEN 'BTO'
             WHEN g.group='tvn_users' THEN 'TVN'
             WHEN g.group='ripple_users' THEN 'RIPPLE'
           END
        ) 
    """
print(sql)


        SELECT 
          platform,
          aging_date AS aging_date_hist,
          industry,
          territory,
          `grouping`,
          collector,    
          customer_no,
          customer_name,
          bucket_id,
          bucket,
          amount,
          prev_amount,
          net_amount,
          net_prev_amount
        FROM `groupfinancial.aging.debtor_aging_by_bucket` 
    


In [ ]:
view = dataset.table(view_name)
table = bigquery.Table(view)
table.view_query = sql
table.view_use_legacy_sql = False

try:
  # delete_table with not_found_ok not working in Airflow bigquery library. 
  # client.delete_table(table, not_found_ok=True)
  try: client.delete_table(table) 
  except: pass

  client.create_table(table)

  print(f'Authorized view created: {dataset_id}.{view_name}')
except:
  raise ValueError(f"Fail to create: {dataset_id}.{view_name}")

Authorized view created: aging.vw_debtor_aging_by_bucket_history


In [ ]:
if IS_DEBUG:
  sql = (
      f"""
        SELECT * FROM `{dataset_id}.{view_name}` 
        LIMIT 5
      """)
  print(sql)
  df = client.query(sql).to_dataframe()
  print(df)


        SELECT * FROM `aging.vw_debtor_aging_by_bucket_history` 
        LIMIT 5
      
  platform aging_date_hist industry  ... prev_amount net_amount net_prev_amount
0     NSTP      2015-04-30   AGENCY  ...           0          0               0
1     NSTP      2015-04-30   AGENCY  ...           0          0               0
2     NSTP      2015-04-30   AGENCY  ...      836.13          0          836.13
3     NSTP      2015-04-30   AGENCY  ...           0     836.13               0
4     NSTP      2015-04-30   AGENCY  ...           0          0               0

[5 rows x 14 columns]


# DEBUG

In [ ]:
# DEBUG only

debug_dataset_id = "trusted-oraclefms"
debug_table_name = ""
df = None

if IS_DEBUG:
  sql = (
      f"""
        SELECT 
        agesum.AGING_DATE aging_date,
        agesum.INDUSTRY industry,
        agesum.TERRITORY territory,
        agesum.GROUPING `grouping`,
        agesum.collector collector,    
        agesum.cust_no,
        cu.customer_name,    
        agesum.amt_due bal_amt,
        agesum.b0_amt b0,
        agesum.b1_amt b1,
        agesum.b2_amt b2,
        agesum.b3_amt b3,
        agesum.b4_amt b4,
        agesum.b5_amt b5,
        agesum.b6_amt b6,
        agesum.cd_amt cd,
        agesum.bg_amt bg
      FROM `mp-vertex.trusted_oraclefms.xxar_agehist_summary` agesum
        INNER JOIN `mp-vertex.trusted_oraclefms.ar_customers` cu
          ON agesum.CUST_NO = cu.customer_number 
      ORDER BY agesum.AGING_DATE, agesum.INDUSTRY;
      """)
  print(sql)
  df = client.query(sql).to_dataframe()
  #print(df)
df